In [1]:
import torch.nn.functional as F
import pandas as pd
from tqdm import trange
from IPython.display import clear_output
import os

import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')
model.to(device)

/Users/vladimirdimitrov/NLP-case/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:01<00:00, 2.80MB/s]
tokenizer.json: 100%|██████████| 17.1M/17.1M [00:56<00:00, 304kB/s]
config.json: 100%|██████████| 694/694 [00:00<00:00, 1.80MB/s]
model.safetensors:   5%|▍         | 52.4M/1.11G [01:55<38:44, 456kB/s]


KeyboardInterrupt: 

In [4]:
df = pd.read_csv('hh_database_clear.csv', index_col=0)
df['query'] = ['query: '] * df.shape[0]
df['Text'] = df['query'] + df['Опыт работы'] + '. ' + df['Последнее/нынешнее место работы'] + '. ' + df['Последняя/нынешняя должность'] + '. ' + df['Образование и ВУЗ'] + '. ' + str(df['Money']) + df['Rate']
list_text = df['Text'].to_list()

In [5]:
BATCH_SIZE = 32
FLAG = False

In [6]:
torch.cuda.empty_cache()
for i in trange(len(list_text) // BATCH_SIZE):
    if os.path.exists('embed_inter.csv') and FLAG:
        dataframe = pd.read_csv('embed_inter.csv', index_col=0)
        i = dataframe.shape[0] // BATCH_SIZE
        FLAG = False
        continue
    if i == 0:
        input_texts = list_text[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)
        dataframe = pd.DataFrame(embeddings.cpu().detach().numpy())
        del embeddings
        print(dataframe.shape)
        torch.cuda.empty_cache()
        continue
    input_texts = list_text[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1) 
    dataframe = pd.concat([dataframe, pd.DataFrame(embeddings.cpu().detach().numpy()).reset_index(drop=True)], ignore_index=True, axis=0)
    print(dataframe.shape)
    if i % 100 == 0:
        clear_output()
        dataframe.to_csv('embed_inter.csv')
    del embeddings
    torch.cuda.empty_cache()
clear_output()
input_texts = list_text[(i+1) * BATCH_SIZE:]
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
dataframe = pd.concat([dataframe, pd.DataFrame(embeddings.cpu().detach().numpy()).reset_index(drop=True)], ignore_index=True, axis=0)

  0%|                                           | 1/694 [00:01<20:19,  1.76s/it]

(64, 768)


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacty of 39.44 GiB of which 149.94 MiB is free. Including non-PyTorch memory, this process has 39.29 GiB memory in use. Of the allocated memory 37.90 GiB is allocated by PyTorch, and 47.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
dataframe.to_csv('embed_e5.csv')